In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
import tensorflow.keras
from tqdm.notebook import tqdm
import torch.nn.functional as F
import matplotlib.pyplot as plt
from IPython.display import Image
import torchvision.transforms as T
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from torchvision.utils import save_image
from scipy.stats import norm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
sample_dir = "thermal"
DATA_DIR = '/content/dataset/'
norms = (.5, .5, .5), (.5, .5, .5)
latent_size=128
epochs = 1
lr_d = 10e-5
lr_g = 10e-4
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
fixed_latent = torch.randn(128, latent_size, 1, 1, device=device)

def denorm(img_tensor):
    return img_tensor * norms[1][0] + norms[0][0]

In [ ]:
transform_low = T.Compose([
    T.Resize((128, 128)),T.CenterCrop(128), T.RandomHorizontalFlip(), T.RandomVerticalFlip(),
    T.ToTensor(),T.Normalize(*norms)
])
transform_mid = T.Compose([
    T.Resize((256, 256)),T.CenterCrop(256), T.RandomHorizontalFlip(), T.RandomVerticalFlip(),
    T.ToTensor(),T.Normalize(*norms)
])

transform_high = T.Compose([
    T.Resize((512, 512)),T.CenterCrop(512), T.RandomHorizontalFlip(), T.RandomVerticalFlip(),
    T.ToTensor(),T.Normalize(*norms)])


In [ ]:
def show_image(train_dl):
    for images,_ in train_dl:
        fig, ax = plt.subplots(figsize=(8,8))
        ax.set_xticks([]); ax.set_yticks([])
        print(images.shape)
        ax.imshow(make_grid(denorm(images.detach()[:32]), nrow=8).permute(1,2,0))
        break

In [ ]:
data = torchvision.datasets.ImageFolder(root=DATA_DIR, transform=transform_mid)
train_dl = DataLoader(data, 128, num_workers=2, pin_memory=True)
show_image(train_dl)